# List of functions

This document describes the main functions used in the Desktop DNS notebooks. These are stored in the 'meshing/' and 'postpro/' directories.

## meshing/

### blade_mesher
Creates a 9-block mesh for a blade profile. The routine first creates an initial coarse mesh to define the block boundaries and then a final mesh is governed by the refinement factor 'refine_fac'.

#### Usage

case = blade_mesher(mesh)

The mesh dictionary contains:
* mesh['case'] path to case directory
* mesh['profile'] path to the blade coordinates file
* mesh['npp'] minimum number of grid points in any direction across a block 
(npp[0] for initial coarse, npp[1] for final mesh)  
* mesh['Lup'] length upstream of the leading-edge setting the inlet plane
* mesh['Ldn'] length downstream of the trailing-edge setting the exit plane
* mesh['Lo'] initial height of the o-grid
* mesh['ywall'] near-wall cell height (ywall[0] for initial coarse, ywall[1] for final mesh)  
* mesh['msmooths'] number of iterations for the Poisson solver ([0] for initial coarse, [1] for final mesh)  
* mesh['scale_factor'] a scaling factor which scales the coordinates equally in both x-y directions
* mesh['refine_fac'] a refinement factor used to increase the number of points for the final mesh
* mesh['flip'] a factor multiplying the y values in order to mirror the geometry
* mesh['cor_fac'] a factor used to shrink cells surrounding the multi-block corner points

The returned case dictionary contains
* case['casename'] path to case directory
* case['blk'] x,y coordinates for each block
* case['next_block'] next_block dictionary defining block connectivity
* case['next_patch'] next_patch dictionary defining patch connectivity
* case['corner'] corner dictionary defining corner connectivity

--- 

### initial_guess
Determines an intial guess of the flow using a compressible potential flow method.

#### Usage 

prop = initial_guess(mesh,geom,bcs,gas,n,nj,Yp)

The bcs dictionary contains:
* bcs['alpha'] inlet flow angle (degrees)
* bcs['vin'] inlet flow velocity (m/s)
* bcs['Toin'] inlet stagnation temperature (K)
* bcs['Poin'] inlet stagnation pressure (Pa)
* bcs['pexit'] exit static pressure

The gas dictionary contains:
* gas['gamma'] ratio of specific heats
* gas['cp'] specific heat at constant pressure (J/kg/K)

The geometry is stored in geom which is an array of dictionaries containing the coordinates:
* geom[ib] contains ['x','y']

n and nj are the streamwise and pitch-wise points used for the initial guess. Yp is an estimate of the loss coefficient.

The interpolated flow properties are returned in prop which is an array of dictionaries containing the flow quantities for each block.
* prop[ib] contains ['ro','ru','rv','rw','Et']


--- 

### read_case
Reads the case files.

#### Usage

case = read_case(casename)

* casename is the path to the case directory.

The returned case dictionary contains
* case['casename'] path to case directory
* case['blk'] x,y coordinates for each block
* case['next_block'] next_block dictionary defining block connectivity
* case['next_patch'] next_patch dictionary defining patch connectivity
* case['corner'] corner dictionary defining corner connectivity
* case['bcs'] dictionary containing the boundary conditions
* case['gas'] dictionary containing the gas properties
* case['solver'] dictionary containing the solver parameters

---

   

## postpro/

### boundarylayer
Extracts the integral paramaters for the suction and pressure surface of a blade.

#### Usage 

bl = boundarylayer(casename,*args):

* casename is the path to the case directory
* if len(args)>0, then the mean flow is read in and the argument arg[0] is taken to be nfiles (see read_2d_mean description). Otherwise the flow is read in from the flow_# files.

The returned bl dictionary contains two nested dictionaries bl['ss'] and bl['ps'] which store the suction and pressure surface quantities:
* bl[:]['x'   ] - x value of surface
* bl[:]['y'   ] - y value of surface
* bl[:]['d0'  ] - boundary layer thickness
* bl[:]['d1'  ] - displacement thickness
* bl[:]['d2'  ] - momentum thickness
* bl[:]['d3'  ] - energy thickness
* bl[:]['cf'  ] - skin friction
* bl[:]['yplus'  ] - y+ of near-wall cell
* bl[:]['xplus'  ] - x+ of near-wall cell
* bl[:]['zplus'  ] - z+ of near-wall cell



--- 

### interp_flo
Interpolates the flow from one case to another, provided the new case directory contains the case files and grid.

#### Usage 

prop = interp_flo(old_case,new_case)

* old_case is a path to the old case directory
* new_case is a path to the new case directory

The returned prop dictionary is the same as that returned by read_flo (see read_flo) and contains the flow properties interpolated onto the new grid.


--- 

### plot_flo
Produces a colour plot of a specified flow property on a spanwise plane.

#### Usage 

plot_flo(prop,geom,plot_var,caxis)

* prop - dictionary containing flow properties (see read_flo)
* geom - dictionary containing grid coordinates (see read_flo)
* plot_var - string identifying flow property - must be an entry in the prop dictionary
* caxis - colour axis range [min_val,max_val]

--- 

### plot_monitor
Produces a plot of the monitor file.

#### Usage 

prop,target =  plot_monitor(casename)
* casename is the path to the case directory
* prop is a dictionary containing 
['time','ro','ru','rv','rw','Et','p','u','v','w','T','s','po','To','mach','vel','alpha']
* target is a dictionary containing the target inlet velocity, angle and entropy
['vin','ain',sin'] 


--- 

### plot_spectrum
Produces a plot of the average power spectrum within blocks defined by the blocks array. The spectrum is computed assuming the blocks can be concatenated in the j-direction, and that the concatenated blocks form a single, periodic flow in the j-direction.

#### Usage 

plot_spectrum(casename,blocks,iexclude)
* casename is the path to the case directory
* blocks is an array defining the blocks to compute the spectrum within - this would usually be [1,2] for the inlet spectrum or [8.9] for the exit spectrum
* iexclude - the first and last iexclude points in the i-direction are not included in the spectrum. The spectrum is averaged over all i and k points within the block and excluding the regions defined by iexclude.
 

--- 

### read_2d_mean
Reads the 'mean2_#_##' files and return the two-dimensional statistics.

#### Usage 

prop,geom,total_time = read_2d_mean(casename,nfiles)

* casename is the path to the case directory
* nfiles defines the number of mean files to read in and therefore the integration time. If nfiles contains a single positive value then the final nfiles are read in. Setting nfiles to a single very large number means that all mean files are read in. If nfiles contains a single negative value then only the mean files identified by the value abs(nfiles) are read in. If nfiles contains two values, the mean files identified by the range nfiles[0]:nfiles[1] are read in.

The returned array of dictionaries prop contains for each block the time average flow quantities derived from Favre-averaging: 

* ['ro','ru','rv','rw','Et','p','u','v','w','T','s','vortz','po','To','mach','mu']

The turbulence properties (tke, turbulence intensity as a percent of inlet velocity, turbulence production, optimum eddy viscosity and strain magnitude).
* ['tke','tu','turb_production','mut_opt','S_']

In addition the time-average entropy budget terms are contained in:
* ['Ds']  $ = \nabla \cdot \overline{(\rho \vec{u} s)}$ the entropy flux derivative
* ['Dsr'] $ = \nabla \cdot \overline{(\vec{q}/T)}$ the heat flux contribution
* ['Dsi'] = Ds + Dsr, the irreversible entropy creation 
* ['dissT'] $ = \overline{(\phi/T)} $ contribution due to resolved dissipation 

The prop structure also contains 
* ['area'] - the cell i-j area 
* ['res'] - the ratio of i-j cell dimension to estimated Kolmogorov length  

The geometry is returned in geom and the integration time in total_time.

--- 

### read_flo
Reads the flow and grid files contained in a case directory.

#### Usage 

prop,geom = read_flo(casename)

* casename is the path to the case directory

The returned array of dictionaries prop contains for each block the flow quantities: 
* ['ro','ru','rv','rw','Et','p','u','v','w','T','s','vortz','po','To','mach','mu']

If there are 'rans_#' files in the directory, these will be read in and the laminar-to-turbulent eddy viscosity will be returned in ['mut_model'].

The returned array of dictionaries geom contains for each block ['x'] and ['y'] values of the grid coordinates.

--- 

### read_kcuts
Reads a range of kcut files and writes an image file of the flow.

#### Usage 

prop,geom = read_kcuts(casename,nfiles,plot_var,caxis):

* casename is the path to the case directory
* nfiles defines the number of cut files to read in. If nfiles contains a single positive value then the final nfiles are read in. Setting nfiles to a single very large number means that all files are read in. If nfiles contains a single negative value then only the files identified by the value abs(nfiles) are read in. If nfiles contains two values, the files identified by the range nfiles[0]:nfiles[1] are read in.
* plot_var is a string identifying the plot variable to show - this must be consistent with the contents of the prop dictionary (see plot_flo and read_flo).
* caxis is the colour axis (see plot_flo)

The returned prop and geom dictionaries contain the flow and geometry relating to the final file read in (see read_flo for contents of prop and geom).

--- 

### read_probe
Reads the probe files in a case directory.

#### Usage 

probe = read_probe(casename)

* casename is the path to the case directory

The returned array of dictionaries probe contains for each probe file
* ['time','ro','ru','rv','rw','Et','u','v','w','p','T','c','po','To','mach','s','vel','alpha','mu','x','y']

The probe location is stored in ['x'] and ['y'].


--- 

### sumvolx
Performs an area integral of a flow property.  

#### Usage 

totx = sumvolx(prop,geom,sum_var,xpos)

* prop is a two-dimensional flow field consistent with the array of dictionaries returned by read_2d_mean.
* geom is the geometry consistent with the array of dictionaries returned by read_2d_mean.
* sum_var is a string identifying the variable to integrate - this must be consistent with the contents of the prop dictionary (see read_2d_mean).
* xpos is an array of values of x defining the limits of integration.

The returned array totx is a 1-d array of length equal to len(xpos) containing the cumulative integral between xpos[0]:xpos[n] where n is in range(len(xpos)).

--- 


### write_mut
Create prescribed eddy viscosity files 'rans_#' using time-average data from a previous simulation. The values of the optimum eddy viscosity are interpolated onto the new grid points.

#### Usage 

write_mut(prop,geom,casename)

* prop is a two-dimensional flow field consistent with the array of dictionaries returned by read_2d_mean.
* geom is the geometry consistent with the array of dictionaries returned by read_2d_mean.
* casename is the path to the <u>new</u> case directory, where the rans files will be written - this directory must contain the new case files and grid.

---

### yp
Extracts the total pressure loss coefficient and wake profile from a time-average flow.

#### Usage 

perf = yp(prop,geom,inlet,exit)

* prop is a two-dimensional flow field consistent with the array of dictionaries returned by read_2d_mean.
* geom is the geometry consistent with the array of dictionaries returned by read_2d_mean.
* inlet is a dictionary containing the array of inlet blocks ['blocks'] and the x location used to determine the inlet conditions ['x'].
* exit is a dictionary containing the array of exit blocks ['blocks'] and the x location used to determine the exit conditions ['x'] and wake profile.

The returned dictionary perf contains

* ['yp'] - the 1d loss coefficient (mass averaged)
* ['poin'] - the computed inlet total pressure (mass averaged)
* ['pin']  - the computed inlet static pressure (area averaged)
* ['poex'] - the computed exit total pressure (mass averaged)
* ['pex']  - the computed exit static pressure (area averaged)
* ['dyn']  - the reference dynamic head used in the definition of loss coefficient - when pex > pin, this is (poin-pin), otherwise it is (poin-pex).
* ['mass in'] - the computed inlet mass flow
* ['mass out'] - the computed exit mass flow
* wake - a dictionary containing the arrays ['y'] and ['yp'] defining the pitchwise position (as a fraction of pitch) and the total pressure loss coefficient.
